##**The Language Model Protein Folder**


Noa Klugman, Yaad Luria, Elad Applbaum, Eyal Zur

**About The Project**

words

#Installs and Imports

In [1]:
%%time

!pip install import-ipynb
import import_ipynb
import os, time
import py3Dmol


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
CPU times: user 159 ms, sys: 15.3 ms, total: 175 ms
Wall time: 9.38 s


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
%cd "/content/drive/MyDrive/Hackathon3D"
import utils

/content/drive/.shortcut-targets-by-id/1qtuhDM-Y1gM5Gxg96zFpbCmuBZD17Z0w/Hackathon3D
importing Jupyter notebook from utils.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
False


Using cache found in /root/.cache/torch/hub/facebookresearch_esm_main


In [4]:
from google.colab import files
import re
import os
import tensorflow as tf
from tensorflow import keras

In [5]:
from google.colab import files
import re
import os
from IPython.display import clear_output

In [6]:
#@title Clone NanoFoldLM trained model


%%bash

if [ ! -f NanoFoldLMReady ]; then
  # install dependencies
  pip -q install biopython
  pip install py3Dmol

  # download model
  if [ ! -d "NanoNet/" ]; then
    git clone https://github.com/YaadLuria/NanoFoldLM --quiet
  fi
  touch NanoFoldLMReady
fi

#Set Parameters

In [7]:
jobname = "test" #@param {type:"string"}
jobname = re.sub(r'\W+', '', jobname)[:50]


download_path = "/src/Downloads/" #@param {type:"string"}

sequence = "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG" #@param {type:"string"}
sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
sequence = re.sub(":+",":",sequence)
sequence = re.sub("^[:]+","",sequence)
sequence = re.sub("[:]+$","",sequence)



amino_acids = set("ACDEFGHIKLMNPQRSTVWY")
for i,aa in enumerate(sequence):
  if aa.upper() not in amino_acids:
    print(f"Invalid amino acid: {aa} in position {i}")

# if utils.check_valid_protein_sequence(sequence):
#   print(f"Invalid amino acid: {aa}")

seqs = sequence.split(":")
lengths = [len(s) for s in seqs]
length = sum(lengths)



download_pdb = False #@param {type:"boolean"}
print_atom_coordinates = False #@param {type:"boolean"}
compare_to_PDB = False #@param {type:"boolean"}
embeding_size = 320 #@param ["320", "480", "640",  "1280"] {type:"raw"}




Compare to solved pdb file

In [8]:
have_solved_structure = True #@param {type:"boolean"}
pdb_id = "6XW6" #@param {type:"string"}
cahin_id = "C" #@param {type:"string"}
if have_solved_structure:
  pdb_test, seq, aa_residues = utils.get_seq_aa_by_id(pdb_id, cahin_id)

Structure exists: './pdb6xw6.ent' 


/usr/local/lib/python3.10/dist-packages/Bio/PDB/Polypeptide.py:144: BiopythonDeprecationWarning: 'three_to_one' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_3to1'.
  warnings.warn(


#Run model

In [9]:
seq_embeddings = utils.get_esm_embedding_for_protein_sequence(sequence, embeding_size)
model_path = "/content/drive/MyDrive/Hackathon3D/Models"
model = tf.keras.models.load_model(f'{model_path}/model_{embeding_size}.tf')
predict = model.predict(seq_embeddings)
utils.matrix_to_pdb(sequence, predict[0], jobname)
# ATOM_LINE = "ATOM{}{}  {}{}{} {}{}{}{}{:.3f}{}{:.3f}{}{:.3f}  1.00{}{:.2f}           {}\n"
# END_LINE = "END\n"
# BACKBONE_ATOMS = ["N", "CA", "C", "O", "CB"]

# k = 1
# coord_matrix = predict[0]
# with open(f"{jobname}.pdb", "w") as pdb_file:
#     for i, aa in enumerate(sequence):
#         third_space = (4 - len(str(i))) * " "
#         for j, atom in enumerate(BACKBONE_ATOMS):
#             if not (aa == "G" and atom == "CB"):  # GLY lacks CB atom
#                 x, y, z = coord_matrix[i][3*j], coord_matrix[i][3*j+1], coord_matrix[i][3*j+2]
#                 b_factor = 0.00
#                 first_space = (7 - len(str(k))) * " "
#                 second_space = (4 - len(atom)) * " "
#                 forth_space = (12 - len("{:.3f}".format(x))) * " "
#                 fifth_space = (8 - len("{:.3f}".format(y))) * " "
#                 sixth_space = (8 - len("{:.3f}".format(z))) * " "
#                 seventh_space = (6 - len("{:.2f}".format(b_factor))) * " "

#                 pdb_file.write(ATOM_LINE.format(first_space, k, atom, second_space, Polypeptide.one_to_three(aa) , "H", third_space,
#                                                 i, forth_space, x, fifth_space, y, sixth_space, z, seventh_space,
#                                                 b_factor, atom[0]))
#                 k += 1

#     pdb_file.write(END_LINE)


Using cache found in /root/.cache/torch/hub/facebookresearch_esm_main


False
False
3/3 [==============================] - 2s 256ms/step


/usr/local/lib/python3.10/dist-packages/Bio/PDB/Polypeptide.py:161: BiopythonDeprecationWarning: 'one_to_three' will be deprecated in a future release of Biopython in favor of 'Bio.PDB.Polypeptide.protein_letters_1to3'.
  warnings.warn(


In [10]:

#@title display (optional) {run: "auto"}

pdb_predict = f"{jobname}.pdb"
# pdb_file_path =  "/content/drive/MyDrive/Hackathon3D/Datasets/Train_set/12E8_1.pdb"


with open(pdb_predict, 'r') as pdb_predict:
  pdb_str = pdb_predict.read()


from string import ascii_uppercase, ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)
pymol_color_list = ["#33ff33","#00ffff","#ff33cc","#ffff00","#ff9999","#e5e5e5","#7f7fff","#ff7f00",
                    "#7fff7f","#199999","#ff007f","#ffdd5e","#8c3f99","#b2b2b2","#007fff","#c4b200",
                    "#8cb266","#00bfbf","#b27f7f","#fcd1a5","#ff7f7f","#ffbfdd","#7fffff","#ffff7f",
                    "#00ff7f","#337fcc","#d8337f","#bfff3f","#ff7fff","#d8d8ff","#3fffbf","#b78c4c",
                    "#339933","#66b2b2","#ba8c84","#84bf00","#b24c66","#7f7f7f","#3f3fa5","#a5512b"]





def show_pdb(pdb_str, show_sidechains=False, show_mainchains=False,
             color="pLDDT", chains=None, vmin=50, vmax=90,
             size=(800,480), hbondCutoff=4.0,
             Ls=None,
             animate=False):

  if chains is None:
    chains = 1 if Ls is None else len(Ls)
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', width=size[0], height=size[1])

  if animate:
    view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  else:
    view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  if color == "pLDDT":
    view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':vmin,'max':vmax}}})
  elif color == "rainbow":
    view.setStyle({'cartoon': {'color':'spectrum'}})
  elif color == "chain":
    for n,chain,color in zip(range(chains),alphabet_list,pymol_color_list):
       view.setStyle({'chain':chain},{'cartoon': {'color':color}})
  if show_sidechains:
    BB = ['C','O','N']
    view.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                  {'sphere':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
    view.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                  {'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  if show_mainchains:
    BB = ['C','O','N','CA']
    view.addStyle({'atom':BB},{'stick':{'colorscheme':f"WhiteCarbon",'radius':0.3}})
  view.zoomTo()
  if animate: view.animate()
  return view

color = "chain" #@param ["confidence", "rainbow", "chain"]
if color == "confidence": color = "pLDDT"
show_sidechains = True #@param {type:"boolean"}
show_mainchains = True #@param {type:"boolean"}
show_pdb(pdb_str, color=color,
         show_sidechains=show_sidechains,
         show_mainchains=show_mainchains,
         Ls=lengths).show()
if(have_solved_structure):
  show_pdb(pdb_test, color=color,
        show_sidechains=show_sidechains,
        show_mainchains=show_mainchains,
        Ls=lengths).show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
#@title Download predicted pdb (optional)
if download_pdb:
  from google.colab import files
  files.download(f"{jobname}.pdb")



